In [2]:
from os.path import join

from rastervision.common.utils import s3_download
from rastervision.common.settings import results_path


run_names = [
    'tagging/7_5_17/ensemble/0',
    'tagging/7_5_17/ensemble/1',
    'tagging/7_5_17/ensemble/2',
    'tagging/7_5_17/ensemble/3',
    'tagging/7_5_17/ensemble/4',
    'tagging/7_11_17/dense_ensemble2/0',
    'tagging/7_11_17/dense_ensemble2/1',
    'tagging/7_11_17/dense_ensemble2/2',
    'tagging/7_11_17/dense_ensemble2/3',
    'tagging/7_11_17/dense_ensemble2/4'
]

test_pred_fn = 'test_preds.csv'

for run_name in run_names:
    s3_download(run_name, test_pred_fn)

['aws', 's3', 'cp', 's3://raster-vision/results/tagging/7_5_17/ensemble/0/test_preds.csv', '/opt/data/results/tagging/7_5_17/ensemble/0/']
['aws', 's3', 'cp', 's3://raster-vision/results/tagging/7_5_17/ensemble/1/test_preds.csv', '/opt/data/results/tagging/7_5_17/ensemble/1/']
['aws', 's3', 'cp', 's3://raster-vision/results/tagging/7_5_17/ensemble/2/test_preds.csv', '/opt/data/results/tagging/7_5_17/ensemble/2/']
['aws', 's3', 'cp', 's3://raster-vision/results/tagging/7_5_17/ensemble/3/test_preds.csv', '/opt/data/results/tagging/7_5_17/ensemble/3/']
['aws', 's3', 'cp', 's3://raster-vision/results/tagging/7_5_17/ensemble/4/test_preds.csv', '/opt/data/results/tagging/7_5_17/ensemble/4/']
['aws', 's3', 'cp', 's3://raster-vision/results/tagging/7_11_17/dense_ensemble2/0/test_preds.csv', '/opt/data/results/tagging/7_11_17/dense_ensemble2/0/']
['aws', 's3', 'cp', 's3://raster-vision/results/tagging/7_11_17/dense_ensemble2/1/test_preds.csv', '/opt/data/results/tagging/7_11_17/dense_ensemble2/

In [10]:
import numpy as np
import math

from rastervision.tagging.data.planet_kaggle import TagStore

run_name = run_names[0]
run_path = join(results_path, run_name)
test_pred_path = join(run_path, test_pred_fn)
test_file_inds = list(TagStore(test_pred_path).file_ind_to_tags.keys())

test_preds_list = []
for run_name in run_names:
    run_path = join(results_path, run_name)
    test_pred_path = join(run_path, test_pred_fn) 
    test_preds = TagStore(test_pred_path).get_tag_array(test_file_inds)
    test_preds_list.append(np.expand_dims(test_preds, axis=2))

test_preds = np.concatenate(test_preds_list, axis=2)
test_preds_sum = np.sum(test_preds, axis=2)
weight_sum = test_preds.shape[2]
true_thresh = math.ceil(weight_sum / 2)
test_preds = test_preds_sum >= true_thresh    

In [11]:
out_path = '/opt/data/jupyter/out/test_preds.csv'
tag_store = TagStore()
for i in range(test_preds.shape[0]):
    tag_store.add_tags(test_file_inds[i], test_preds[i, :])

tag_store.save(out_path)